In [1]:
import dspy
from dspy.predict import Retry
from dspy.primitives.assertions import assert_transform_module, backtrack_handler
import importlib
import json

importlib.reload(dspy)

# Configure the AzureOpenAI language model
azure_turbo = dspy.AzureOpenAI(api_base='https://bionlp-gpt4-wang.openai.azure.com/', api_key="a494edc84d714b6c8a12e7212974b793", api_version='2024-03-01-preview', model='gpt-35-turbo-16k')
dspy.settings.configure(lm=azure_turbo)

/opt/anaconda3/envs/dspy_repo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class PubMedQA(dspy.Signature):
    question = dspy.InputField(desc="Question")
    context = dspy.InputField(desc="A list of facts which provide the necessary context to answer the question. You should directly mention the portions of the facts that you used to help answer the question.")
    options = dspy.InputField(desc="Options, you should select one of them based on whichever is correct.")
    answer = dspy.OutputField(desc="Your final answer should contain only one of YES, MAYBE, or NO (all in upper case) based on whichever option is correct")


class PubMedQA_Response(dspy.Module):
    """Use the provided context to answer a question by providing a step-by-step explanation and also respond with one of YES, MAYBE, or NO (all in upper case) based on whichever is correct"""
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(PubMedQA, n=5)
        self.compare_answers = dspy.MultiChainComparison(PubMedQA, M=5)

    def forward(self, question, context, options):
        response = self.generate_answer(context=context, question=question, options=options, config=dict(temperature=1))

        while len(response.completions.answer) < 5:

            curr_len = len(response.completions.answer)

            for i in range(curr_len):
                response.completions.answer.append(response.completions.answer[i])
                response.completions.rationale.append(response.completions.rationale[i])

                if len(response.completions.answer) == 5:
                    break
        
        best_response = self.compare_answers(context=context, question=question, options=options, completions=response.completions)
        
        valid_response = "YES" in best_response.answer or "MAYBE" in best_response.answer or "NO" in best_response.answer
        dspy.Suggest(valid_response, "Your final answer must be one of YES, MAYBE, or NO (all upper case) as part of your final answer")
        
        return dspy.Prediction(answer=best_response.answer, rationale=response.rationale)

In [3]:
import re

def eval_metric_pubmedqa(true, prediction, trace=None):
    try:
        pred = prediction.answer
        matches = re.findall(r"\b(YES|NO|MAYBE)\b", pred)
        parsed_answer = matches[-1] if matches else ""
        return parsed_answer == true.answer
    except:
        return False

In [4]:
import random

random.seed(42)

# Add the training set here:
with open("/Users/khandekarns/Documents/pubmedqa/data/pqaa_train_set.json") as file:
    train_set = json.load(file)

dspy_train_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in train_set.items():

    combined_context = ""

    for i, context in enumerate(train_set[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": train_set[key]['QUESTION'], "options": options, "context": combined_context, "answer": train_set[key]['final_decision'].upper()}).with_inputs("question", "context", "options") 

    dspy_train_set_pubmedqa.append(example)

random.shuffle(dspy_train_set_pubmedqa)

dspy_train_set_pubmedqa = random.sample(dspy_train_set_pubmedqa, 5000)



In [5]:
import json 


with open("/Users/khandekarns/Documents/pubmedqa/data/test_set.json") as file:
    test_set = json.load(file)


with open("/Users/khandekarns/Documents/pubmedqa/data/test_ground_truth.json") as file:
    gt = json.load(file)


dspy_test_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in test_set.items():

    combined_context = ""

    for i, context in enumerate(test_set[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": test_set[key]['QUESTION'], "options": options, "context": combined_context, "answer": gt[key].upper()}).with_inputs("question", "context", "options") 
    dspy_test_set_pubmedqa.append(example)

In [6]:
import json
import random

random.seed(42)


with open("/Users/khandekarns/Documents/pubmedqa/data/ori_pqal.json") as file:
    labeled_set = json.load(file)

val_set = {}

for key, value in labeled_set.items():
    if key not in gt:
        val_set[key] = labeled_set[key]
       

# Augment validation dataset here: 

with open("/Users/khandekarns/Documents/pubmedqa/data/test_set.json") as file:
    test_set = json.load(file)


with open("/Users/khandekarns/Documents/pubmedqa/data/test_ground_truth.json") as file:
    gt = json.load(file)


dspy_val_set_pubmedqa = []

options = "1. YES\n2. MAYBE\n3. NO"

for key, val in val_set.items():

    combined_context = ""

    for i, context in enumerate(val_set[key]['CONTEXTS']):
        
        index_str = str(i + 1)

        combined_context += f"{index_str}. {context}\n"

    example = dspy.Example({"question": labeled_set[key]['QUESTION'], "options": options, "context": combined_context, "answer": labeled_set[key]['final_decision'].upper()}).with_inputs("question", "context", "options") 

    dspy_val_set_pubmedqa.append(example)

random.shuffle(dspy_val_set_pubmedqa)

In [7]:
from dspy.evaluate import Evaluate

evaluate_test = Evaluate(devset=dspy_test_set_pubmedqa, metric=eval_metric_pubmedqa, num_threads=6, display_progress=True, display_table=True)

In [8]:
answerqa_with_assertions = assert_transform_module(PubMedQA_Response().map_named_predictors(Retry), backtrack_handler) 
evaluate_test(answerqa_with_assertions)

Average Metric: 2 / 2  (100.0):   0%|          | 2/500 [02:07<7:33:39, 54.66s/it]  INFO:backoff:Backing off request(...) for 0.4s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.4 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1154eaac0> with kwargs {'n': 1, 'temperature': 0.0, 'max_tokens': 75}


ERROR:backoff:Giving up request(...) after 2 tries (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})
ERROR:dspy.evaluate.evaluate:2024-07-25T15:33:23.927788Z [error    ] Error for example in dev set: 		 Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}} [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
Average Metric: 2.0 / 3  

Backing off 1.0 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1154eaac0> with kwargs {'temperature': 0.7}


ERROR:backoff:Giving up request(...) after 2 tries (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})
ERROR:dspy.evaluate.evaluate:2024-07-25T15:36:24.819335Z [error    ] Error for example in dev set: 		 Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}} [dspy.evaluate.evaluate] filename=evaluate.py lineno=180
Average Metric: 2.0 / 4

...

Backing off 0.8 seconds after 2 tries calling function <function AzureOpenAI.request at 0x1154eaac0> with kwargs {'n': 5, 'temperature': 1}


ERROR:backoff:Giving up request(...) after 3 tries (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})
INFO:backoff:Backing off request(...) for 0.3s (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded call rate limit of your current OpenAI S0 pricing tier. Please retry after 10 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


Backing off 0.3 seconds after 1 tries calling function <function AzureOpenAI.request at 0x1154eaac0> with kwargs {'n': 5, 'temperature': 1}


KeyboardInterrupt: 

ERROR:backoff:Giving up request(...) after 2 tries (openai.RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-03-01-preview have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 60 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}})


In [ ]:
from tqdm import tqdm
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

tqdm._instances.clear()

config = dict(max_bootstrapped_demos=3, max_labeled_demos=0, num_candidate_programs=20, num_threads=6)
teleprompter = BootstrapFewShotWithRandomSearch(metric = eval_metric_pubmedqa, **config)
answer_question_teacher = teleprompter.compile(student=assert_transform_module(PubMedQA_Response().map_named_predictors(Retry), backtrack_handler), teacher = assert_transform_module(PubMedQA_Response().map_named_predictors(Retry), backtrack_handler), trainset=dspy_train_set_pubmedqa, valset=dspy_val_set_pubmedqa)

In [ ]:
evaluate_test(answer_question_teacher)